Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Tejas Yeshwantrao Wandile"
COLLABORATORS = ""

---

# You cannot use Pandas! I will deduct points after manual check if you use Pandas. You CANNOT use the 'csv' module to read the file


# Hint: Ensure to strip all strings so there is no space in them

# DO NOT use StudentID from the non_nomralized table. Let the normalized table  automatically handle StudentID. 

In [1]:
from IPython.display import display, HTML
import pandas as pd
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows


In [2]:
conn = create_connection('non_normalized.db')
sql_statement = "select * from Students;"
df = pd.read_sql_query(sql_statement, conn)
display(df)

,StudentID,Name,Degree,Exams,Scores
0,1,"Rodriguez, Pamela",graduate,"exam7 (2017), exam9 (2018), exam3 (2018), exam...","61, 38, 85, 70, 44, 43, 68"
1,2,"Jackson, Kristie",undergraduate,"exam2 (2017), exam2 (2017), exam8 (2018), exam...","66, 69, 85, 62, 92, 72, 44, 50, 23"
2,3,"Curtis, George",graduate,"exam7 (2017), exam6 (2017), exam2 (2017), exam...","68, 73, 62, 72"
3,4,"Casey, Christie",undergraduate,"exam10 (2019), exam4 (2019), exam4 (2019), exa...","41, 52, 52, 70"
4,5,"Yoder, Emily",undergraduate,"exam4 (2019), exam4 (2019), exam4 (2019)","52, 45, 61"
...,...,...,...,...,...
95,96,"Dyer, Benjamin",undergraduate,"exam10 (2019), exam4 (2019), exam6 (2017), exa...","47, 53, 64, 57, 71, 79, 77, 56"
96,97,"Delgado, Jason",undergraduate,"exam1 (2016), exam8 (2018)","72, 83"
97,98,"Cortez, Kenneth",undergraduate,"exam3 (2018), exam5 (2020), exam6 (2017)","83, 55, 71"
98,99,"Short, Elizabeth",undergraduate,"exam3 (2018), exam9 (2018), exam3 (2018), exam...","87, 34, 84, 75, 80, 60, 48, 55, 75"


In [30]:
def normalize_database(non_normalized_db_filename):
#     Normalize 'non_normalized.db'
#     Call the normalized database 'normalized.db'
#     Function Output: No outputs
#     Requirements:
#     Create four tables
#     Degrees table has one column:
#         [Degree] column is the primary key
    
#     Exams table has two columns:
#         [Exam] column is the primary key column
#         [Year] column stores the exam year
    
#     Students table has four columns:
#         [StudentID] primary key column 
#         [First_Name] stores first name
#         [Last_Name] stores last name
#         [Degree] foreign key to Degrees table
        
#     StudentExamScores table has four columns:
#         [PK] primary key column,
#         [StudentID] foreign key to Students table,
#         [Exam] foreign key to Exams table ,
#         [Score] exam score
    from pprint import pprint
    import pandas

    # open db file for non-normalized db
    conn = create_connection(non_normalized_db_filename)
    # get cursor
    cur = conn.cursor()
    cur.execute("select * from Students;")
    original_table = cur.fetchall()
    conn.close()
    #pprint(original_table)
    
    # open db file for normalized db
    conn_nor = create_connection('normalized.db', True)
    # get cursor
    cur_nor = conn_nor.cursor()
    
    cur_nor.executescript("""
    DROP TABLE IF EXISTS DEGREES;
    DROP TABLE IF EXISTS EXAMS;
    DROP TABLE IF EXISTS STUDENTS;
    DROP TABLE IF EXISTS STUDENTEXAMSCORES;
    
    PRAGMA foreign_keys = ON;
    
    CREATE TABLE DEGREES (
            Degree TEXT PRIMARY KEY
            );
        
    CREATE TABLE EXAMS (
            Exam TEXT  NOT NULL PRIMARY KEY,
            Year INTEGER64  NOT NULL
            
        );
        
    CREATE TABLE STUDENTS (
            StudentID INTEGER  NOT NULL PRIMARY KEY,
            First_Name TEXT  NOT NULL,
            Last_Name TEXT  NOT NULL,
            Degree TEXT NOT NULL,
            FOREIGN KEY(Degree) REFERENCES DEGREES(Degree)
    );
    
    CREATE TABLE STUDENTEXAMSCORES (
            PK INTEGER  NOT NULL PRIMARY KEY,
            StudentID INTEGER  NOT NULL,
            Exam TEXT  NOT NULL,
            scores VARCHAR  NOT NULL,
            FOREIGN KEY(StudentID) REFERENCES STUDENTS(StudentID),
            FOREIGN KEY(Exam) REFERENCES EXAMS(Exam)
            
    );
    
    
    """)
    arr_exam = []
    arr_year = []
    arr_degree = []
    for _, _, degree, *_ in original_table:
        #print(degree)
        if degree not in arr_degree:
            arr_degree.append(degree)
            cur_nor.execute("INSERT INTO DEGREES (degree) VALUES(?)",
                      (degree,))


    for _, _, _, exam, _ in original_table:
        exam_no = exam.replace('(','').replace(')','').split(',')
        for val in exam_no:
            exam, year = val.split()
            if exam not in arr_exam:
                arr_exam.append(exam)
                arr_year.append(int(year))

    for i in range(len(arr_exam)):
        cur_nor.execute("INSERT INTO EXAMS (EXAM, YEAR) VALUES(?,?)",
                         (arr_exam[i].strip(),int(arr_year[i])))

    for _, name, degree, *_ in original_table:
        last_name, first_name = name.split()

        cur_nor.execute("INSERT INTO STUDENTS (first_name, last_name, degree) VALUES(?,?,?)",
                    (first_name, last_name.replace(',',''), degree))
    Sc=[]
    for StudentId, _, _, Exam, Scores in original_table:
        #print(len(Exam.split(',')))
        exam_no = Exam.replace('(','').replace(')','').split(',')
        i=0
        for val in exam_no:
            exam, year = val.split()
            Sc = Scores.split(',')
            cur_nor.execute("INSERT INTO STUDENTEXAMSCORES (STUDENTID, EXAM, Scores) VALUES(?,?,?)",(StudentId, exam.strip(), int(Sc[i])))
            i = i + 1
    conn_nor.commit()
    conn.close()

In [31]:
normalize_database('non_normalized.db')
conn = create_connection('normalized.db')


In [32]:
def ex1():
    # Write an SQL statement that SELECTs all rows from the `Exams` table and sort the exams by Year
    # output columns: exam, year
    sql_statement = "select * from EXAMS order by year, exam"
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [33]:
sql_statement = ex1()
data = pd.read_csv("ex1.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Exam,Year
0,exam1,2016
1,exam2,2017
2,exam6,2017
3,exam7,2017
4,exam3,2018
5,exam8,2018
6,exam9,2018
7,exam10,2019
8,exam4,2019
9,exam5,2020


In [34]:
def ex2():
    # Write an SQL statement that SELECTs all rows from the `Degrees` table and sort the degrees by name
    # output columns: degree
    
    sql_statement = "select * from Degrees order by Degree"
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [35]:
sql_statement = ex2()
data = pd.read_csv("ex2.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Degree
0,graduate
1,undergraduate


In [36]:
def ex3():
    # Write an SQL statement that counts the numbers of gradate and undergraduate students
    # output columns: degree, count_degree
    
    sql_statement = "select Degree, count(*) as count_degree from Students group by Degree"
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [37]:
sql_statement = ex3()
data = pd.read_csv("ex3.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Degree,count_degree
0,graduate,35
1,undergraduate,65


In [38]:
def ex4():
    # Write an SQL statement that calculates the exam averages for exams; sort by average in descending order.
    # output columns: exam, year, average
    # round to two decimal places
    
    
    sql_statement = "select e.exam, e.year, round(avg(s.scores),2) as average from Exams e, STUDENTEXAMSCORES s where e.exam=s.exam group by e.exam order by average desc;"
    #sql_statement = "select round(avg(scores),2) from STUDENTEXAMSCORES where exam='exam7';"
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [39]:
sql_statement = ex4()
data = pd.read_csv("ex4.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Exam,Year,average
0,exam3,2018,86.16
1,exam8,2018,80.18
2,exam1,2016,75.09
3,exam7,2017,68.96
4,exam2,2017,64.28
5,exam6,2017,61.80
6,exam4,2019,54.04
7,exam5,2020,45.56
8,exam10,2019,44.68
9,exam9,2018,39.43


In [40]:
sql_statement = "select e.exam, e.year, s.scores from studentexamscores s INNER JOIN Exams e USING(Exam) WHERE e.Exam='exam3';"
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
display(df)

,Exam,Year,scores
0,exam3,2018,85
1,exam3,2018,92
2,exam3,2018,89
3,exam3,2018,95
4,exam3,2018,87
5,exam3,2018,77
6,exam3,2018,84
7,exam3,2018,81
8,exam3,2018,81
9,exam3,2018,91


In [41]:
def ex5():
    # Write an SQL statement that calculates the exam averages for degrees; sort by average in descending order.
    # output columns: degree, average 
    # round to two decimal places
    
    sql_statement = "select s.Degree, round(avg(sc.scores),2) as average from studentexamscores sc, students s where s.studentid=sc.studentid group by s.Degree;"
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [42]:
sql_statement = ex5()
data = pd.read_csv("ex5.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Degree,average
0,graduate,62.50
1,undergraduate,61.63


In [43]:
def ex6():
    # Write an SQL statement that calculates the exam averages for students; sort by average in descending order. 
    # Show only top 10 students
    # output columns: first_name, last_name, degree, average
    # round to two decimal places
    # Order by average in descending order
    # Warning two of the students have the same average!!!
    
    sql_statement = "select s.First_Name, s.Last_Name, s.Degree, round(avg(sc.scores),2) as average from Students s, STUDENTEXAMSCORES sc where s.studentid=sc.studentid group by sc.studentid order by average desc limit 10;"
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [44]:
sql_statement = ex6()
data = pd.read_csv("ex6.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,First_Name,Last_Name,Degree,average
0,John,Washington,graduate,83.00
1,Robert,Andrade,undergraduate,82.00
2,Paul,Smith,undergraduate,79.00
3,Jason,Delgado,undergraduate,77.50
4,Calvin,Perez,undergraduate,75.00
5,Jason,Thompson,graduate,75.00
6,Calvin,Martinez,undergraduate,74.50
7,Billy,Palmer,undergraduate,74.38
8,Matthew,King,undergraduate,73.56
9,Debra,Pratt,undergraduate,72.60
